# Feature Engineering (adding sentiment scores, normalizing data & adding new features)

## After testing lots of sentiment analysis models, we are going to be using "ROBERTA MODEL" as it prevails over all the others (As of now in 2024). 

In [56]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)


# "TESTING OUR ROBERTA MODEL"

In [57]:


# testing our sentiment analysis function
def sentiment_analysis(tweet):
# precprcess tweet
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)


    labels = ['Negative', 'Neutral', 'Positive']

    # sentiment analysis
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
        
        l = labels[i]
        s = scores[i]
        print(l,s)

    # Calculate compound score
    compound_score = np.dot(scores, [-1, 0, 1])
    print(f"Compound Score: {compound_score:.4}")

sentiment_analysis("testing out our wondefull  tweet, with roberta model")


Negative 0.012825871
Neutral 0.73450035
Positive 0.25267383
Compound Score: 0.2398


## Importing our Cleaned NYT Data

In [74]:
# Define the file paths
file_path = 'Final_NYT_articles.csv'

# Read the CSV files
NYT_articles = pd.read_csv(file_path)

NYT_articles.tail()
NYT_articles.insert(0, 'article_count', range(1, len(NYT_articles) + 1))
NYT_articles.tail()





,article_count,pub_date,headline,abstract,lead_paragraph,byline,section_name,web_url,source,multimedia_url
144,145,2024-06-01T11:00:06+0000,Oil and Gas Companies Are Trying to Rig the Marketplace,"Fossil fuel interests are spreading misinformation that renewable energy is harmful, unreliable and worse for consumers.",Many of us focused on the problem of climate change have been waiting for the day when renewable energy would become cheaper than fossil fuels.,By Andrew Dessler,Opinion,https://www.nytimes.com/2024/06/01/opinion/clean-energy-solar-wind.html,The New York Times,https://www.nytimes.com/images/2024/06/03/opinion/01dessler/01dessler-articleLarge.jpg
145,146,2024-07-07T19:55:06+0000,"Officials Urge Coastal Texans to Evacuate, but Some Are Unfazed","Despite warnings that Beryl could be a “deadly storm,” many local residents decided to stay put, gambling that they could survive its wind and rains.","As Beryl chugged toward the Texas Gulf Coast on Sunday, oil workers fled drilling platforms, tourist towns battered by previous storms shut down their ferries, and state officials urged people to evacuate at-risk low-lying coastal areas.","By Edgar Sandoval, Maria Jimenez Moya and Jack Healy",U.S.,https://www.nytimes.com/2024/07/07/us/texas-evacuations-storm-beryl.html,The New York Times,https://www.nytimes.com/images/2024/07/07/multimedia/07hurricane-beryl-unfazed-bkqz/07hurricane-beryl-unfazed-bkqz-articleLarge.jpg
146,147,2024-07-07T09:00:16+0000,New Plan to Target Russia’s Oil Revenue Brings Debate in White House,Treasury officials want to impose penalties on tankers that help Russian oil evade sanctions. White House aides worry that risks making gasoline more expensive.,Officials in President Biden’s Treasury Department have proposed new actions aimed at crippling a fleet of aging oil tankers that are helping deliver Russian oil to buyers around the world in defiance of Western sanctions.,By Jim Tankersley and Alan Rappeport,U.S.,https://www.nytimes.com/2024/07/07/us/politics/russia-oil-ukraine-shadow-fleet.html,The New York Times,https://www.nytimes.com/images/2024/06/25/multimedia/00dc-russiafleet-vkwb/00dc-russiafleet-vkwb-articleLarge.jpg
147,148,2024-07-06T10:00:05+0000,"Facing New ‘Greenwashing’ Law, an Oil Industry Website Goes Dark",Oil sands companies pushing a carbon capture project shut down their website after a law banning misleading environmental claims was passed.,"When Parliament passed a law last month banning misleading or false environmental claims in advertising, or “greenwashing,” the reaction from an alliance of six oil sands companies was swift. The website for their Pathways Alliance, pushing a carbon capture and storage plan for oil sands emissions, more or less vanished. Most of the companies scrubbed their own websites and social media pages of everything related to environmental issues.",By Ian Austen,World,https://www.nytimes.com/2024/07/06/world/canada/canada-greenwashing-oil-sands.html,The New York Times,https://www.nytimes.com/images/2024/07/06/multimedia/06canadaletter-oilsands1-gmcz/06canadaletter-oilsands1-gmcz-articleLarge.jpg
148,149,2024-07-02T01:23:39+0000,Judge Orders Biden Administration to Resume Permits for Gas Exports,"President Biden had paused new natural gas export terminals to assess their effects on the climate, economy and national security. A federal judge disagreed.","A federal judge on Monday ordered the Biden administration to resume issuing permits for new liquefied natural gas export facilities after the government had paused that process in January to analyze how those exports affect climate change, the economy and national security.",By Coral Davenport,Climate,https://www.nytimes.com/2024/07/01/climate/federal-judge-biden-gas-exports.html,The New York Times,https://www.nytimes.com/images/2024/07/01/multimedia/01cli-lngpause-kpzj/01cli-lngpause-kpzj-articleLarge-v4.jpg


## Creating the function to add sentiment scores

In [60]:
example = NYT_articles["headline"][4]
print(f"Example: {example}")
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative(sentiment_score_headline)' : scores[0],
        'neutral(sentiment_score_headline)' : scores[1],
        'positive(sentiment_score_headline)' : scores[2],
        'compound(sentiment_score_headline)' : np.dot(scores, [-1, 0, 1]) 
    }
    return scores_dict

polarity_scores_roberta(example)

Example: White House Said to Delay Decision on Enormous Natural Gas Export Terminal


{'negative(sentiment_score_headline)': 0.19727981,
 'neutral(sentiment_score_headline)': 0.78225654,
 'positive(sentiment_score_headline)': 0.020463653,
 'compound(sentiment_score_headline)': -0.1768161579966545}

# In NYT data we have lots of text columns, we need to go over some examples to see which column should we use for our sentiment scores.

In [61]:
pd.set_option('display.max_colwidth', None) 
NYT_articles.iloc[4][['article_count', 'headline', 'abstract', 'lead_paragraph']]

article_count                                                                                                                                                                                                                                                                                                                                      5
headline                                                                                                                                                                                                                                                                  White House Said to Delay Decision on Enormous Natural Gas Export Terminal
abstract                                                                                                                                                                                              Before deciding whether to approve it, the Energy Department will analyze the climate impacts of CP2, one of 17 proposed

# As we can see the columns have different text sizes as expected lets run the sentiment scores on each column and objectively choose which column is better to keep track of sentiment scores.

In [62]:
example = NYT_articles["headline"][4]
example2 = NYT_articles["abstract"][4]
example3 = NYT_articles["lead_paragraph"][4]

def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative(sentiment_score_headline)' : scores[0],
        'neutral(sentiment_score_headline)' : scores[1],
        'positive(sentiment_score_headline)' : scores[2],
        'compound(sentiment_score_headline)' : np.dot(scores, [-1, 0, 1]) 
    }
    return scores_dict
print(f"Example: {example}")
print(polarity_scores_roberta(example))

print(f"Example: {example2}")
print(polarity_scores_roberta(example2))

print(f"Example: {example3}")
print(polarity_scores_roberta(example3))

Example: White House Said to Delay Decision on Enormous Natural Gas Export Terminal
{'negative(sentiment_score_headline)': 0.19727981, 'neutral(sentiment_score_headline)': 0.78225654, 'positive(sentiment_score_headline)': 0.020463653, 'compound(sentiment_score_headline)': -0.1768161579966545}
Example: Before deciding whether to approve it, the Energy Department will analyze the climate impacts of CP2, one of 17 proposed LNG export terminals.
{'negative(sentiment_score_headline)': 0.09672564, 'neutral(sentiment_score_headline)': 0.87878036, 'positive(sentiment_score_headline)': 0.02449399, 'compound(sentiment_score_headline)': -0.07223165221512318}
Example: The Biden administration is pausing a decision on whether to approve what would be the largest natural gas export terminal in the United States, a delay that could stretch past the November election and spell trouble for that project and 16 other proposed terminals, according to three people with knowledge of the matter.
{'negative(s

## The sentiment score of lead paragraph appears to be much more negative.

## Let's do a different article 

In [63]:
example = NYT_articles["headline"][6]
example2 = NYT_articles["abstract"][6]
example3 = NYT_articles["lead_paragraph"][6]

print(f"Example: {example}")
print(polarity_scores_roberta(example))

print(f"Example: {example2}")
print(polarity_scores_roberta(example2))

print(f"Example: {example3}")
print(polarity_scores_roberta(example3))

Example: To Slash Carbon Emissions, Colleges Are Digging Really Deep
{'negative(sentiment_score_headline)': 0.13952605, 'neutral(sentiment_score_headline)': 0.7351717, 'positive(sentiment_score_headline)': 0.12530227, 'compound(sentiment_score_headline)': -0.01422378420829773}
Example: A growing number of colleges and universities are using deep underground pipes to heat and cool their buildings without burning fossil fuels.
{'negative(sentiment_score_headline)': 0.03893204, 'neutral(sentiment_score_headline)': 0.732312, 'positive(sentiment_score_headline)': 0.22875598, 'compound(sentiment_score_headline)': 0.1898239403963089}
Example: When administrators at Princeton University decided to cut the carbon emissions that came from heating and cooling their campus, they opted for a method that is gaining popularity among colleges and universities.
{'negative(sentiment_score_headline)': 0.010158518, 'neutral(sentiment_score_headline)': 0.47147632, 'positive(sentiment_score_headline)': 0.51

In [64]:
example = NYT_articles["headline"][8]
example2 = NYT_articles["abstract"][8]
example3 = NYT_articles["lead_paragraph"][8]


print(f"Example: {example}")
print(polarity_scores_roberta(example))

print(f"Example: {example2}")
print(polarity_scores_roberta(example2))

print(f"Example: {example3}")
print(polarity_scores_roberta(example3))

Example: Ukraine Targets Russian Oil Plants, Aiming to Disrupt Military Operations
{'negative(sentiment_score_headline)': 0.75273335, 'neutral(sentiment_score_headline)': 0.24250716, 'positive(sentiment_score_headline)': 0.0047594877, 'compound(sentiment_score_headline)': -0.7479738621041179}
Example: Kyiv hit an oil depot on Friday, the latest in recent attacks on such facilities as it seeks to inflict damage away from the largely deadlocked front line.
{'negative(sentiment_score_headline)': 0.8099761, 'neutral(sentiment_score_headline)': 0.18588692, 'positive(sentiment_score_headline)': 0.0041369763, 'compound(sentiment_score_headline)': -0.8058391246013343}
Example: Ukraine hit an oil depot in Russia in a drone attack on Friday, officials on both sides said, the latest in a series of recent assaults targeting Russian oil facilities as Kyiv increasingly seeks to strike critical infrastructure behind Russian lines.
{'negative(sentiment_score_headline)': 0.76570624, 'neutral(sentiment_

# In this example the sentiment score is more or less the same. 

In [65]:
example = NYT_articles["headline"][13]
example2 = NYT_articles["abstract"][13]
example3 = NYT_articles["lead_paragraph"][13]


print(f"Example: {example}")
print(polarity_scores_roberta(example))

print(f"Example: {example2}")
print(polarity_scores_roberta(example2))

print(f"Example: {example3}")
print(polarity_scores_roberta(example3))

Example: Biden Administration to Fine Oil and Gas Companies for Excess Methane
{'negative(sentiment_score_headline)': 0.18190868, 'neutral(sentiment_score_headline)': 0.79285306, 'positive(sentiment_score_headline)': 0.025238229, 'compound(sentiment_score_headline)': -0.15667045302689075}
Example: A plan to impose a fee of $900 to $1,500 on every excess ton of methane emissions would be the first federal price on greenhouse gas pollution.
{'negative(sentiment_score_headline)': 0.31265682, 'neutral(sentiment_score_headline)': 0.6591446, 'positive(sentiment_score_headline)': 0.028198589, 'compound(sentiment_score_headline)': -0.2844582311809063}
Example: The Biden administration announced new moves on Friday to curb the release from oil and gas facilities of methane, a potent greenhouse gas that is responsible for more than a quarter of the warming the planet is currently experiencing.
{'negative(sentiment_score_headline)': 0.25499666, 'neutral(sentiment_score_headline)': 0.6819029, 'pos

## Both 3 compund scores are same as well. 

In [66]:
example = NYT_articles["headline"][20]
example2 = NYT_articles["abstract"][20]
example3 = NYT_articles["lead_paragraph"][20]


print(f"Example: {example}")
print(polarity_scores_roberta(example))

print(f"Example: {example2}")
print(polarity_scores_roberta(example2))

print(f"Example: {example3}")
print(polarity_scores_roberta(example3))

Example: Missing Profits May Be a Problem for the Green Transition
{'negative(sentiment_score_headline)': 0.65399086, 'neutral(sentiment_score_headline)': 0.32308462, 'positive(sentiment_score_headline)': 0.022924501, 'compound(sentiment_score_headline)': -0.6310663633048534}
Example: The inability to generate acceptable returns is impeding the switch to renewable energy.
{'negative(sentiment_score_headline)': 0.724043, 'neutral(sentiment_score_headline)': 0.2592742, 'positive(sentiment_score_headline)': 0.01668281, 'compound(sentiment_score_headline)': -0.7073602024465799}
Example: In October, the International Energy Agency issued a brief flare of optimism when it suggested that global carbon emissions from fossil fuels may already have peaked.
{'negative(sentiment_score_headline)': 0.025541592, 'neutral(sentiment_score_headline)': 0.3825491, 'positive(sentiment_score_headline)': 0.5919093, 'compound(sentiment_score_headline)': 0.5663676969707012}


# "Missing Profits May Be a Problem for the Green Transition" this is negative compund for the green energy, however this is positive for the crude oil....

## There are two things we can do: 

### 1)Filter out the Articles or future tweets with the word containing green. 
### or 
### 2)If we have a negative compound for the green energy, we can do the opposise of -0.63 to +0.63 for that article. 

In [67]:
# Define the words you are searching for
keywords = r'green|renewable'

# Filter rows where these words appear in the 'headline', 'abstract', or 'lead_paragraph'
filtered_rows = NYT_articles[NYT_articles[['headline', 'abstract', 'lead_paragraph']].apply(
    lambda x: x.str.contains(keywords, case=False, na=False).any(), axis=1)]

# Print the filtered rows with selected columns
filtered_rows[['article_count', 'headline', 'abstract', 'lead_paragraph']]


,article_count,headline,abstract,lead_paragraph
13,14,Biden Administration to Fine Oil and Gas Companies for Excess Methane,"A plan to impose a fee of $900 to $1,500 on every excess ton of methane emissions would be the first federal price on greenhouse gas pollution.","The Biden administration announced new moves on Friday to curb the release from oil and gas facilities of methane, a potent greenhouse gas that is responsible for more than a quarter of the warming the planet is currently experiencing."
20,21,Missing Profits May Be a Problem for the Green Transition,The inability to generate acceptable returns is impeding the switch to renewable energy.,"In October, the International Energy Agency issued a brief flare of optimism when it suggested that global carbon emissions from fossil fuels may already have peaked."
21,22,U.S. Carbon Emissions Fell in 2023 as Coal Use Tumbled to New Lows,"The drop was big, but emissions will need to fall three times as fast for the rest of the decade if the country wants to meet its climate goals.","America’s greenhouse gas emissions fell 1.9 percent in 2023, in large part because the burning of coal to produce electricity plummeted to its lowest level in half a century, according to estimates published on Wednesday by the Rhodium Group, a nonpartisan research firm."
43,44,"BP to Increase Oil Output, New Chief Says","Murray Auchincloss signaled that he would take a more profit-oriented approach than his predecessor, who started a big push into renewables.","BP’s new chief executive, Murray Auchincloss, promised a flexible approach to the shift away from fossil fuels as the oil giant reported a $3 billion profit in its latest quarter on Tuesday."
52,53,"‘Garbage Lasagna’: Dumps Are a Big Driver of Warming, Study Says","Decades of buried trash is releasing methane, a powerful greenhouse gas, at higher rates than previously estimated, the researchers said.","They’re vast expanses that can be as big as towns: open landfills where household waste ends up, whether it’s vegetable scraps or old appliances."
66,67,"Oil Fields Release Far More Methane Than Thought, Study Finds","In parts of New Mexico, more than 9 percent of all natural gas produced goes into the atmosphere, where it acts as a powerful greenhouse gas.","Oil and gas producers in major oil fields across the United States may be emitting three times as much planet-warming methane gas as official estimates, according to new research published Wednesday, the latest study to suggest that emissions from the fossil fuel sector may be grossly undercounted."
75,76,Corn to Power Airplanes? Biden Administration Sets a High Bar.,"Producers of biofuels like ethanol, which could help create a new generation of jet fuel, would have to overhaul their practices to receive tax credits.","In a move aimed at lowering the greenhouse gas emissions of air travel, the Biden administration on Tuesday issued new guidelines for how fuel producers — and in particular, makers of ethanol from corn — could qualify for tax credits under a plan to increase the supply of so-called sustainable aviation fuel."
107,108,How China Rose to Lead the World in Cars and Solar Panels,"Heavy subsidies for industry, together with weak sales in China, have set the stage for an export boom, raising fears of factory job losses elsewhere.","For decades, China has moved methodically to dominate ever more industries, from toys and clothing in the 1980s to semiconductors and renewable energy today. China now produces a third of the world’s manufactured goods — more than the United States, Germany, Japan, South Korea and Britain combined. Its trade surplus in these goods is equal to a tenth of the entire Chinese economy."
121,122,"Oil Companies Expand Offshore Drilling, Pointing to Energy Needs",Shell and others say they plan to drill for oil and gas in the Gulf of Mexico in part because doing so releases fewer greenhouse gases than drilling on land.,"About 80 miles southeast of Louisiana’

In [68]:
example = NYT_articles["headline"][20]
example2 = NYT_articles["abstract"][20]
example3 = NYT_articles["lead_paragraph"][20]


print(f"Example: {example}")
print(polarity_scores_roberta(example))

print(f"Example: {example2}")
print(polarity_scores_roberta(example2))

print(f"Example: {example3}")
print(polarity_scores_roberta(example3))

Example: Missing Profits May Be a Problem for the Green Transition
{'negative(sentiment_score_headline)': 0.65399086, 'neutral(sentiment_score_headline)': 0.32308462, 'positive(sentiment_score_headline)': 0.022924501, 'compound(sentiment_score_headline)': -0.6310663633048534}
Example: The inability to generate acceptable returns is impeding the switch to renewable energy.
{'negative(sentiment_score_headline)': 0.724043, 'neutral(sentiment_score_headline)': 0.2592742, 'positive(sentiment_score_headline)': 0.01668281, 'compound(sentiment_score_headline)': -0.7073602024465799}
Example: In October, the International Energy Agency issued a brief flare of optimism when it suggested that global carbon emissions from fossil fuels may already have peaked.
{'negative(sentiment_score_headline)': 0.025541592, 'neutral(sentiment_score_headline)': 0.3825491, 'positive(sentiment_score_headline)': 0.5919093, 'compound(sentiment_score_headline)': 0.5663676969707012}


We are essentially dealing with contextual sentiment, where the sentiment's impact on crude oil depends on the article's subject matter. If an article positively discusses green energy, it may imply negative sentiment for crude oil, even if the overall sentiment is positive. Here’s how you can handle this:

1. **Topic Classification:**
   - First, classify the article into categories like "Crude Oil," "Green Energy," or "Other." This way, you can identify whether the article primarily discusses topics related to oil or green energy.
   - Use NLP models like BERT or LDA (Latent Dirichlet Allocation) for topic modeling or classification.

2. **Custom Sentiment Mapping:**
   - After topic classification, you can define custom sentiment rules:
     - For articles classified under "Green Energy," if the sentiment is positive, map it as negative for crude oil.
     - For articles under "Crude Oil," positive sentiment should remain positive for crude oil.

3. **Sentiment Recalibration:**
   - Once you have the article's topic, adjust the sentiment scores based on the category. For instance:
     - Positive sentiment for "Green Energy" → Negative sentiment for crude oil.
     - Positive sentiment for "Crude Oil" → Positive sentiment for crude oil.



## Writing Code so that it smoothly adds to the whole dataset

In [75]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm

# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

# Function to calculate sentiment scores
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative(sentiment_score_headline)': scores[0],
        'neutral(sentiment_score_headline)': scores[1],
        'positive(sentiment_score_headline)': scores[2],
        'compound(sentiment_score_headline)': np.dot(scores, [-1, 0, 1])
    }
    return scores_dict

# Function to classify article as 'Crude Oil', 'Green Energy', or 'Other'
def classify_article(row):
    text = row['headline'] + " " + row['abstract']
    
    # Ensure keywords related to energy topics are detected
    if any(keyword in text.lower() for keyword in ['oil', 'crude oil', 'oil price', 'saudi aramco', 'opec', 'fossil fuel']):
        return 'Crude Oil'
    elif any(keyword in text.lower() for keyword in ['green', 'green energy', 'renewable', 'solar', 'wind', 'clean energy']):
        return 'Green Energy'
    else:
        return 'Other'

# Adjust sentiment based on the topic of the article and invert the sentiment for Green Energy
def adjust_sentiment_based_on_topic(sentiment_dict, topic):
    print(f"\nOriginal Sentiment: {sentiment_dict}")
    print(f"Article Topic: {topic}")
    
    # If the article is about Green Energy, invert the compound sentiment score
    if topic == 'Green Energy':
        print("Inverting sentiment because the topic is Green Energy.")
        
        # Invert the compound score (simply multiply by -1)
        sentiment_dict['compound(sentiment_score_headline)'] = -sentiment_dict['compound(sentiment_score_headline)']
    
    print(f"Adjusted Sentiment: {sentiment_dict}")
    return sentiment_dict

# DataFrame to store results
res = {}

# Iterate over articles and calculate sentiment scores
for i, row in tqdm(NYT_articles.iterrows(), total=len(NYT_articles)):
    try:
        text = row['headline']
        myid = row['article_count']
        
        # Step 1: Get sentiment scores
        sentiment_scores = polarity_scores_roberta(text)
        
        # Step 2: Classify the article topic
        topic = classify_article(row)
        print(f"Article {myid}: '{row['headline']}' classified as {topic}")
        
        # Step 3: Adjust sentiment based on the topic
        sentiment_scores = adjust_sentiment_based_on_topic(sentiment_scores, topic)
        
        # Store results
        res[myid] = sentiment_scores
    except RuntimeError:
        print(f'Broke for id {myid}')

# Convert results into DataFrame
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'article_count'})
results_df = results_df.merge(NYT_articles, how='left')

# Display final DataFrame
results_df.head()


  1%|▏         | 2/149 [00:00<00:11, 12.97it/s]

Article 1: 'Saudi Aramco Abruptly Drops Plans to Expand Oil Production' classified as Crude Oil

Original Sentiment: {'negative(sentiment_score_headline)': 0.36805832, 'neutral(sentiment_score_headline)': 0.6016179, 'positive(sentiment_score_headline)': 0.030323852, 'compound(sentiment_score_headline)': -0.33773447200655937}
Article Topic: Crude Oil
Adjusted Sentiment: {'negative(sentiment_score_headline)': 0.36805832, 'neutral(sentiment_score_headline)': 0.6016179, 'positive(sentiment_score_headline)': 0.030323852, 'compound(sentiment_score_headline)': -0.33773447200655937}
Article 2: 'Where Southerners Go to Fill the Tank and Feed the Family' classified as Other

Original Sentiment: {'negative(sentiment_score_headline)': 0.16693851, 'neutral(sentiment_score_headline)': 0.7786358, 'positive(sentiment_score_headline)': 0.05442567, 'compound(sentiment_score_headline)': -0.11251284182071686}
Article Topic: Other
Adjusted Sentiment: {'negative(sentiment_score_headline)': 0.16693851, 'neut

  4%|▍         | 6/149 [00:00<00:10, 13.76it/s]

Article 4: 'Shipping Costs Soar in Wake of Red Sea Attacks' classified as Other

Original Sentiment: {'negative(sentiment_score_headline)': 0.418735, 'neutral(sentiment_score_headline)': 0.5609675, 'positive(sentiment_score_headline)': 0.020297507, 'compound(sentiment_score_headline)': -0.39843749068677425}
Article Topic: Other
Adjusted Sentiment: {'negative(sentiment_score_headline)': 0.418735, 'neutral(sentiment_score_headline)': 0.5609675, 'positive(sentiment_score_headline)': 0.020297507, 'compound(sentiment_score_headline)': -0.39843749068677425}
Article 5: 'White House Said to Delay Decision on Enormous Natural Gas Export Terminal' classified as Other

Original Sentiment: {'negative(sentiment_score_headline)': 0.19727981, 'neutral(sentiment_score_headline)': 0.78225654, 'positive(sentiment_score_headline)': 0.020463653, 'compound(sentiment_score_headline)': -0.1768161579966545}
Article Topic: Other
Adjusted Sentiment: {'negative(sentiment_score_headline)': 0.19727981, 'neutral(se

  5%|▌         | 8/149 [00:00<00:10, 13.75it/s]

...

 98%|█████████▊| 146/149 [00:09<00:00, 14.86it/s]

Article 145: 'Oil and Gas Companies Are Trying to Rig the Marketplace' classified as Crude Oil

Original Sentiment: {'negative(sentiment_score_headline)': 0.6490368, 'neutral(sentiment_score_headline)': 0.3367188, 'positive(sentiment_score_headline)': 0.014244453, 'compound(sentiment_score_headline)': -0.6347923716530204}
Article Topic: Crude Oil
Adjusted Sentiment: {'negative(sentiment_score_headline)': 0.6490368, 'neutral(sentiment_score_headline)': 0.3367188, 'positive(sentiment_score_headline)': 0.014244453, 'compound(sentiment_score_headline)': -0.6347923716530204}
Article 146: 'Officials Urge Coastal Texans to Evacuate, but Some Are Unfazed' classified as Green Energy

Original Sentiment: {'negative(sentiment_score_headline)': 0.4788941, 'neutral(sentiment_score_headline)': 0.49697405, 'positive(sentiment_score_headline)': 0.024131842, 'compound(sentiment_score_headline)': -0.4547622725367546}
Article Topic: Green Energy
Inverting sentiment because the topic is Green Energy.
Adju

100%|██████████| 149/149 [00:09<00:00, 14.97it/s]

Article 148: 'Facing New ‘Greenwashing’ Law, an Oil Industry Website Goes Dark' classified as Crude Oil

Original Sentiment: {'negative(sentiment_score_headline)': 0.6550137, 'neutral(sentiment_score_headline)': 0.32957393, 'positive(sentiment_score_headline)': 0.015412432, 'compound(sentiment_score_headline)': -0.6396012483164668}
Article Topic: Crude Oil
Adjusted Sentiment: {'negative(sentiment_score_headline)': 0.6550137, 'neutral(sentiment_score_headline)': 0.32957393, 'positive(sentiment_score_headline)': 0.015412432, 'compound(sentiment_score_headline)': -0.6396012483164668}
Article 149: 'Judge Orders Biden Administration to Resume Permits for Gas Exports' classified as Other

Original Sentiment: {'negative(sentiment_score_headline)': 0.10071805, 'neutral(sentiment_score_headline)': 0.85983014, 'positive(sentiment_score_headline)': 0.039451733, 'compound(sentiment_score_headline)': -0.06126631796360016}
Article Topic: Other
Adjusted Sentiment: {'negative(sentiment_score_headline)

,article_count,negative(sentiment_score_headline),neutral(sentiment_score_headline),positive(sentiment_score_headline),compound(sentiment_score_headline),pub_date,headline,abstract,lead_paragraph,byline,section_name,web_url,source,multimedia_url
0,1,0.368058,0.601618,0.030324,-0.337734,2024-01-30T13:40:11+0000,Saudi Aramco Abruptly Drops Plans to Expand Oil Production,"The pullback, at the behest of the Saudi government and made with little comment, probably reflects a more subdued outlook for demand of Saudi Arabia’s oil.","Saudi Aramco said Tuesday that it would call off plans to expand its oil output, a remarkable turnaround by one of the world’s leading petroleum producers.",By Stanley Reed,Business Day,https://www.nytimes.com/2024/01/30/business/saudi-aramco-oil-production.html,The New York Times,https://www.nytimes.com/images/2024/01/30/multimedia/30aramco-01-pgwm/30aramco-01-pgwm-articleLarge.jpg
1,2,0.166939,0.778636,0.054426,-0.112513,2024-01-29T10:01:56+0000,Where Southerners Go to Fill the Tank and Feed the Family,Are they gas stations that serve food or restaurants that pump gas? A new photography book explores the lure of these restorative community rest stops.,New York City has its bodegas. The South has its gas stations.,By Kim Severson,Food,https://www.nytimes.com/2024/01/29/dining/southern-gas-station-restaurants.html,The New York Times,https://www.nytimes.com/images/2024/01/31/multimedia/31Gas-Station-Food4-vqpm/29Gas-Station-Food4-vqpm-articleLarge.jpg
2,3,0.505036,0.465767,0.029197,-0.475839,2024-01-27T08:00:40+0000,Democrats Say Biden Hasn’t ‘Made the Case’ on Climate Despite Achievements,The president likes to talk about his climate agenda as an opportunity to create jobs. Polling shows that may be the wrong way to win over voters.,"President Biden has done more than any president to tackle climate change, but strategists are grappling with an uncomfortable fact: Voters don’t seem to know it.",By Lisa Friedman,Climate,https://www.nytimes.com/2024/01/27/climate/biden-climate-campaign.html,The New York Times,https://www.nytimes.com/images/2024/01/26/multimedia/00cli-bidenclimatepitch-01-hpfc/00cli-bidenclimatepitch-01-hpfc-articleLarge.jpg
3,4,0.418735,0.560968,0.020298,-0.398437,2024-01-24T19:59:53+0000,Shipping Costs Soar in Wake of Red Sea Attacks,"Ships risking Houthi attacks must pay high insurance premiums, while vessels avoiding the region lose time going around Africa.","For about two months, a barrage of missile and drone attacks in the Red Sea by Houthi militants has posed a difficult choice to shipowners using the Suez Canal: risk an airborne strike and pay sharply higher insurance rates, or forgo the canal and take the longer route around Africa, snarling schedules and entailing higher fuel charges.",By Stanley Reed,Business Day,https://www.nytimes.com/2024/01/24/business/red-sea-attacks-shipping-costs.html,The New York Times,https://www.nytimes.com/images/2024/01/23/multimedia/00redsea-shipping-zgjk/00redsea-shipping-zgjk-articleLarge.jpg
4,5,0.197280,0.782257,0.020464,-0.176816,2024-01-24T17:59:40+0000,White House Said to Delay Decision on Enormous Natural Gas Export Terminal,"Before deciding whether to approve it, the Energy Department will analyze the climate impacts of CP2, one of 17 proposed LNG export terminals.","The Biden administration is pausing a decision on whether to approve what would be the largest natural gas export terminal in the United States, a delay that could stretch past the November election and spell trouble for that project and 16 other proposed terminals, according to three people with knowledge of the matter.",By Coral Davenport,Climate,https://www.nytimes.com/2024/01/24/climate/biden-lng-export-terminal-cp2.html,The New York Times,https://www.nytimes.com/images/2024/01/24/multimedia/24cli-gasfight-02-lfzw/24cli-gasfight-02-lfzw-articleLarge.jpg


## As we can see below the article that talks about the Green Energy is negative now. As good for Green means bad for crude oil.

In [76]:
results_df.iloc[20]

article_count                                                                                                                                                                                             21
negative(sentiment_score_headline)                                                                                                                                                                  0.653991
neutral(sentiment_score_headline)                                                                                                                                                                   0.323085
positive(sentiment_score_headline)                                                                                                                                                                  0.022925
compound(sentiment_score_headline)                                                                                                                                                  

In [13]:
# import pandas as pd
# import numpy as np
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from scipy.special import softmax
# from tqdm import tqdm
# res = {}
# for i, row in tqdm(NYT_articles.iterrows(), total=len(NYT_articles)):
#     try:
#         text = row['headline']
#         myid = row['article_count']
#         res[myid] = polarity_scores_roberta(text)
#     except RuntimeError:
#         print(f'Broke for id {myid}')


  0%|          | 0/149 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 149/149 [00:09<00:00, 15.27it/s]


In [15]:
# results_df = pd.DataFrame(res).T
# results_df = results_df.reset_index().rename(columns={'index': 'article_count'})
# results_df = results_df.merge(NYT_articles, how='left')
# results_df.tail(3)

,article_count,negative(sentiment_score_headline),neutral(sentiment_score_headline),positive(sentiment_score_headline),compound(sentiment_score_headline),pub_date,headline,abstract,lead_paragraph,byline,section_name,web_url,source,multimedia_url
146,147,0.279863,0.699984,0.020153,-0.259711,2024-07-07T09:00:16+0000,New Plan to Target Russia’s Oil Revenue Brings Debate in White House,Treasury officials want to impose penalties on tankers that help Russian oil evade sanctions. White House aides worry that risks making gasoline more expensive.,Officials in President Biden’s Treasury Department have proposed new actions aimed at crippling a fleet of aging oil tankers that are helping deliver Russian oil to buyers around the world in defiance of Western sanctions.,By Jim Tankersley and Alan Rappeport,U.S.,https://www.nytimes.com/2024/07/07/us/politics/russia-oil-ukraine-shadow-fleet.html,The New York Times,https://www.nytimes.com/images/2024/06/25/multimedia/00dc-russiafleet-vkwb/00dc-russiafleet-vkwb-articleLarge.jpg
147,148,0.655014,0.329574,0.015412,-0.639601,2024-07-06T10:00:05+0000,"Facing New ‘Greenwashing’ Law, an Oil Industry Website Goes Dark",Oil sands companies pushing a carbon capture project shut down their website after a law banning misleading environmental claims was passed.,"When Parliament passed a law last month banning misleading or false environmental claims in advertising, or “greenwashing,” the reaction from an alliance of six oil sands companies was swift. The website for their Pathways Alliance, pushing a carbon capture and storage plan for oil sands emissions, more or less vanished. Most of the companies scrubbed their own websites and social media pages of everything related to environmental issues.",By Ian Austen,World,https://www.nytimes.com/2024/07/06/world/canada/canada-greenwashing-oil-sands.html,The New York Times,https://www.nytimes.com/images/2024/07/06/multimedia/06canadaletter-oilsands1-gmcz/06canadaletter-oilsands1-gmcz-articleLarge.jpg
148,149,0.100718,0.859830,0.039452,-0.061266,2024-07-02T01:23:39+0000,Judge Orders Biden Administration to Resume Permits for Gas Exports,"President Biden had paused new natural gas export terminals to assess their effects on the climate, economy and national security. A federal judge disagreed.","A federal judge on Monday ordered the Biden administration to resume issuing permits for new liquefied natural gas export facilities after the government had paused that process in January to analyze how those exports affect climate change, the economy and national security.",By Coral Davenport,Climate,https://www.nytimes.com/2024/07/01/climate/federal-judge-biden-gas-exports.html,The New York Times,https://www.nytimes.com/images/2024/07/01/multimedia/01cli-lngpause-kpzj/01cli-lngpause-kpzj-articleLarge-v4.jpg


In [ ]:
# import pandas as pd
# import numpy as np
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from scipy.special import softmax
# from tqdm import tqdm

# # Load the model and tokenizer
# model_name = "cardiffnlp/twitter-roberta-base-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)

# # Define function to calculate polarity scores for abstracts
# def polarity_scores_roberta_abstract(example):
#     encoded_text = tokenizer(example, return_tensors='pt')
#     output = model(**encoded_text)
#     scores = output[0][0].detach().numpy()
#     scores = softmax(scores)
#     scores_dict = {
#         'negative(sentiment_score_abstract)': scores[0],
#         'neutral(sentiment_score_abstract)': scores[1],
#         'positive(sentiment_score_abstract)': scores[2],
#         'compound(sentiment_score_abstract)': np.dot(scores, [-1, 0, 1])
#     }
#     return scores_dict

# # Function to classify article as 'Crude Oil', 'Green Energy', or 'Other'
# def classify_article(row):
#     text = row['headline'] + " " + row['abstract']
#     section = row['section_name']
    
#     # Classify based on keywords and section names
#     if 'Business' in section or 'Climate' in section:
#         if any(keyword in text.lower() for keyword in ['oil', 'crude oil', 'oil price', 'saudi aramco', 'opec', 'fossil fuel']):
#             return 'Crude Oil'
#         elif any(keyword in text.lower() for keyword in ['green energy', 'renewable', 'solar', 'wind', 'clean energy']):
#             return 'Green Energy'
#     # If no clear match, return Other
#     return 'Other'

# # Adjust sentiment based on the topic of the article
# def adjust_sentiment_based_on_topic(sentiment_dict, topic):
#     # If the article is about Green Energy, invert the positive/negative sentiment for Crude Oil
#     if topic == 'Green Energy':
#         sentiment_dict['positive(sentiment_score_abstract)'], sentiment_dict['negative(sentiment_score_abstract)'] = \
#             sentiment_dict['negative(sentiment_score_abstract)'], sentiment_dict['positive(sentiment_score_abstract)']
#         sentiment_dict['compound(sentiment_score_abstract)'] = -sentiment_dict['compound(sentiment_score_abstract)']
#     return sentiment_dict

# # Calculate sentiment scores for abstracts and apply filtering
# res_abstract = {}
# for i, row in tqdm(results_df.iterrows(), total=len(results_df)):
#     try:
#         text = row['abstract']
#         myid = row['article_count']
        
#         # Step 1: Get sentiment scores for the abstract
#         sentiment_scores = polarity_scores_roberta_abstract(text)
        
#         # Step 2: Classify the article topic based on headline and abstract
#         topic = classify_article(row)
        
#         # Step 3: Adjust sentiment based on the topic
#         sentiment_scores = adjust_sentiment_based_on_topic(sentiment_scores, topic)
        
#         # Store results
#         res_abstract[myid] = sentiment_scores
#     except RuntimeError:
#         print(f'Broke for id {myid}')

# # Convert the results dictionary to a DataFrame
# results_df_abstract = pd.DataFrame(res_abstract).T.reset_index().rename(columns={'index': 'article_count'})

# # Ensure article_count is the same type in both DataFrames
# results_df['article_count'] = results_df['article_count'].astype(str)
# results_df_abstract['article_count'] = results_df_abstract['article_count'].astype(str)

# # Merge the new DataFrame with the original one
# results_df_final = results_df.merge(results_df_abstract, on='article_count', how='left')

# # Display the head of the final DataFrame
# results_df_final.head()


In [16]:
# import pandas as pd
# import numpy as np
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from scipy.special import softmax
# from tqdm import tqdm

# # Load the model and tokenizer
# model_name = "cardiffnlp/twitter-roberta-base-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)

# # Define function to calculate polarity scores for abstracts
# def polarity_scores_roberta_abstract(example):
#     encoded_text = tokenizer(example, return_tensors='pt')
#     output = model(**encoded_text)
#     scores = output[0][0].detach().numpy()
#     scores = softmax(scores)
#     scores_dict = {
#         'negative(sentiment_score_abstract)': scores[0],
#         'neutral(sentiment_score_abstract)': scores[1],
#         'positive(sentiment_score_abstract)': scores[2],
#         'compound(sentiment_score_abstract)': np.dot(scores, [-1, 0, 1])
#     }
#     return scores_dict

# # Load the results DataFrame (assuming it's named results_df)
# # results_df = pd.read_csv('path_to_your_csv_file.csv') # Uncomment and set the correct path to your file

# # Calculate sentiment scores for abstracts and store in a dictionary
# res_abstract = {}
# for i, row in tqdm(results_df.iterrows(), total=len(results_df)):
#     try:
#         text = row['abstract']
#         myid = row['article_count']
#         res_abstract[myid] = polarity_scores_roberta_abstract(text)
#     except RuntimeError:
#         print(f'Broke for id {myid}')

# # Convert the results dictionary to a DataFrame
# results_df_abstract = pd.DataFrame(res_abstract).T.reset_index().rename(columns={'index': 'article_count'})

# # Ensure article_count is the same type in both DataFrames
# results_df['article_count'] = results_df['article_count'].astype(str)
# results_df_abstract['article_count'] = results_df_abstract['article_count'].astype(str)

# # Merge the new DataFrame with the original one
# results_df_final = results_df.merge(results_df_abstract, on='article_count', how='left')

# # Display the head of the final DataFrame
# results_df_final.head()


100%|██████████| 149/149 [00:12<00:00, 11.71it/s]


,article_count,negative(sentiment_score_headline),neutral(sentiment_score_headline),positive(sentiment_score_headline),compound(sentiment_score_headline),pub_date,headline,abstract,lead_paragraph,byline,section_name,web_url,source,multimedia_url,negative(sentiment_score_abstract),neutral(sentiment_score_abstract),positive(sentiment_score_abstract),compound(sentiment_score_abstract)
0,1,0.368058,0.601618,0.030324,-0.337734,2024-01-30T13:40:11+0000,Saudi Aramco Abruptly Drops Plans to Expand Oil Production,"The pullback, at the behest of the Saudi government and made with little comment, probably reflects a more subdued outlook for demand of Saudi Arabia’s oil.","Saudi Aramco said Tuesday that it would call off plans to expand its oil output, a remarkable turnaround by one of the world’s leading petroleum producers.",By Stanley Reed,Business Day,https://www.nytimes.com/2024/01/30/business/saudi-aramco-oil-production.html,The New York Times,https://www.nytimes.com/images/2024/01/30/multimedia/30aramco-01-pgwm/30aramco-01-pgwm-articleLarge.jpg,0.425521,0.555503,0.018976,-0.406545
1,2,0.166939,0.778636,0.054426,-0.112513,2024-01-29T10:01:56+0000,Where Southerners Go to Fill the Tank and Feed the Family,Are they gas stations that serve food or restaurants that pump gas? A new photography book explores the lure of these restorative community rest stops.,New York City has its bodegas. The South has its gas stations.,By Kim Severson,Food,https://www.nytimes.com/2024/01/29/dining/southern-gas-station-restaurants.html,The New York Times,https://www.nytimes.com/images/2024/01/31/multimedia/31Gas-Station-Food4-vqpm/29Gas-Station-Food4-vqpm-articleLarge.jpg,0.037574,0.847687,0.114738,0.077164
2,3,0.505036,0.465767,0.029197,-0.475839,2024-01-27T08:00:40+0000,Democrats Say Biden Hasn’t ‘Made the Case’ on Climate Despite Achievements,The president likes to talk about his climate agenda as an opportunity to create jobs. Polling shows that may be the wrong way to win over voters.,"President Biden has done more than any president to tackle climate change, but strategists are grappling with an uncomfortable fact: Voters don’t seem to know it.",By Lisa Friedman,Climate,https://www.nytimes.com/2024/01/27/climate/biden-climate-campaign.html,The New York Times,https://www.nytimes.com/images/2024/01/26/multimedia/00cli-bidenclimatepitch-01-hpfc/00cli-bidenclimatepitch-01-hpfc-articleLarge.jpg,0.690003,0.280284,0.029713,-0.660291
3,4,0.418735,0.560968,0.020298,-0.398437,2024-01-24T19:59:53+0000,Shipping Costs Soar in Wake of Red Sea Attacks,"Ships risking Houthi attacks must pay high insurance premiums, while vessels avoiding the region lose time going around Africa.","For about two months, a barrage of missile and drone attacks in the Red Sea by Houthi militants has posed a difficult choice to shipowners using the Suez Canal: risk an airborne strike and pay sharply higher insurance rates, or forgo the canal and take the longer route around Africa, snarling schedules and entailing higher fuel charges.",By Stanley Reed,Business Day,https://www.nytimes.com/2024/01/24/business/red-sea-attacks-shipping-costs.html,The New York Times,https://www.nytimes.com/images/2024/01/23/multimedia/00redsea-shipping-zgjk/00redsea-shipping-zgjk-articleLarge.jpg,0.638661,0.350388,0.010952,-0.627709
4,5,0.197280,0.782257,0.020464,-0.176816,2024-01-24T17:59:40+0000,White House Said to Delay Decision on Enormous Natural Gas Export Terminal,"Before deciding whether to approve it, the Energy Department will analyze the climate impacts of CP2, one of 17 proposed LNG export terminals.","The Biden administration is pausing a decision on whether to approve what would be the largest natural gas export terminal in the United States, a delay that could stretch past the November election and spell trouble for that project and 16 other proposed terminals, according to three people with knowledge of the matter.",By Coral Davenport,Climate,https://www.nytimes.com/2024/01/24/climate/biden-l

In [77]:
NYT_articles_results = results_df

# Save the merged DataFrame to a new CSV file
output_path = 'NYT_articles_results.csv'
NYT_articles_results.to_csv(output_path, index=False)

print(f"The merged DataFrame has been saved to {output_path}")

The merged DataFrame has been saved to NYT_articles_results.csv


# Now we are doing the same for the NYT

In [56]:
file_path = 'Final_X_articles.csv'

# Read the CSV files
X_articles = pd.read_csv(file_path)

X_articles.tail()

,tweet_count,tweet.created_at,tweet.user.name,tweet.user.location,tweet.user.followers_count,tweet.text,tweet.view_count,tweet.favorite_count,tweet.retweet_count,tweet.reply_count,tweet.media
1726,1727,2024-07-09 18:17:04+00:00,Asia Fact Check Lab,NaN,148,"A claim emerged among Chinese netizens in June that Saudi Arabia terminated a 50-year formal agreement with the U.S. to conduct oil transactions in dollars, under a deal called the “petrodollar agreement.”\n\nBut are the claims true: https://t.co/j6hooZcmV0 https://t.co/ZNDay8bMoj",66,3,1,0,"[{'display_url': 'pic.x.com/znday8bmoj', 'expanded_url': 'https://twitter.com/AFCL_eng/status/1810739964075999403/photo/1', 'id_str': '1810739960112369665', 'indices': [256, 279], 'media_key': '3_1810739960112369665', 'media_url_https': 'https://pbs.twimg.com/media/GSEKZmcaUAEbSeh.jpg', 'type': 'photo', 'url': 'https://t.co/ZNDay8bMoj', 'ext_media_availability': {'status': 'Available'}, 'features': {'large': {'faces': [{'x': 147, 'y': 299, 'h': 45, 'w': 45}]}, 'medium': {'faces': [{'x': 147, 'y': 299, 'h': 45, 'w': 45}]}, 'small': {'faces': [{'x': 92, 'y': 188, 'h': 28, 'w': 28}]}, 'orig': {'faces': [{'x': 147, 'y': 299, 'h': 45, 'w': 45}]}}, 'sizes': {'large': {'h': 1080, 'w': 1080, 'resize': 'fit'}, 'medium': {'h': 1080, 'w': 1080, 'resize': 'fit'}, 'small': {'h': 680, 'w': 680, 'resize': 'fit'}, 'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}}, 'original_info': {'height': 1080, 'width': 1080, 'focus_rects': [{'x': 0, 'y': 0, 'w': 1080, 'h': 605}, {'x': 0, 'y': 0, 'w': 1080, 'h': 1080}, {'x': 0, 'y': 0, 'w': 947, 'h': 1080}, {'x': 0, 'y': 0, 'w': 540, 'h': 1080}, {'x': 0, 'y': 0, 'w': 1080, 'h': 1080}]}, 'media_results': {'result': {'media_key': '3_1810739960112369665'}}}]"
1727,1728,2024-07-09 18:28:45+00:00,House Appropriations,"Washington, DC",22922,The FY25 Interior and Environment Appropriations Act supports an all-of-the-above energy strategy: \n→ Promotes U.S. energy independence\n→ Requires onshore and offshore oil and gas lease sales\n→ Expands access to critical minerals\n→ Blocks onerous fees and regulations https://t.co/HwiDzCEW02,1787,9,2,4,"[{'display_url': 'pic.x.com/hwidzcew02', 'expanded_url': 'https://twitter.com/HouseAppropsGOP/status/1810742905470120438/video/1', 'id_str': '1810742434982416389', 'indices': [268, 291], 'media_key': '7_1810742434982416389', 'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1810742434982416389/pu/img/k7-Ya33Ixf70WAdv.jpg', 'type': 'video', 'url': 'https://t.co/HwiDzCEW02', 'additional_media_info': {'monetizable': False}, 'ext_media_availability': {'status': 'Available'}, 'sizes': {'large': {'h': 1080, 'w': 1080, 'resize': 'fit'}, 'medium': {'h': 1080, 'w': 1080, 'resize': 'fit'}, 'small': {'h': 680, 'w': 680, 'resize': 'fit'}, 'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}}, 'original_info': {'height': 1080, 'width': 1080, 'focus_rects': []}, 'video_info': {'aspect_ratio': [1, 1], 'duration_millis': 23600, 'variants': [{'content_type': 'application/x-mpegURL', 'url': 'https://video.twimg.com/ext_tw_video/1810742434982416389/pu/pl/3sWVtyEtEtVuQy7d.m3u8?tag=12'}, {'bitrate': 432000, 'content_type': 'video/mp4', 'url': 'https://video.twimg.com/ext_tw_video/1810742434982416389/pu/vid/avc1/320x320/JqmuJaxdPhTDC13S.mp4?tag=12'}, {'bitrate': 832000, 'content_type': 'video/mp4', 'url': 'https://video.twimg.com/ext_tw_video/1810742434982416389/pu/vid/avc1/540x540/2eVtvl67CDUjri7_.mp4?tag=12'}, {'bitrate': 1280000, 'content_type': 'video/mp4', 'url': 'https://video.twimg.com/ext_tw_video/1810742434982416389/pu/vid/avc1/720x720/vdO_-PF1Ba_ahkDk.mp4?tag=12'}]}, 'media_results': {'result': {'media_key': '7_1810742434982416389'}}}]"
1728,1729,2024-07-09 18:45:17+00:00,EIA,"Washington, DC",181421,☝️Correction: #Cryptocurrency mining. Apologies for the typo.,2245,1,0,0,NaN
1729,1730,2024-07-09 19:46:22+00:00,GetWireless,Minnesota,693,"💡100% of the top 20 U.S. oil and gas producers trust Semtech AirLink® router solutions to ensure remote dr

In [46]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm

# Load your sentiment analysis model and tokenizer (same as before)
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Function to calculate sentiment scores for tweets
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative(sentiment_score_tweet)': scores[0],
        'neutral(sentiment_score_tweet)': scores[1],
        'positive(sentiment_score_tweet)': scores[2],
        'compound(sentiment_score_tweet)': np.dot(scores, [-1, 0, 1])
    }
    return scores_dict

# Function to classify tweet as 'Crude Oil', 'Green Energy', or 'Other'
def classify_tweet(row):
    text = row['tweet.text']
    
    # Classify based on keywords
    if any(keyword in text.lower() for keyword in ['oil', 'crude oil', 'brent', 'fossil fuel']):
        return 'Crude Oil'
    elif any(keyword in text.lower() for keyword in ['green energy', 'renewable', 'solar', 'wind', 'clean energy']):
        return 'Green Energy'
    else:
        return 'Other'

# Adjust sentiment based on the topic of the tweet
def adjust_sentiment_based_on_topic(sentiment_dict, topic):
    # If the tweet is about Green Energy, invert the positive/negative sentiment for Crude Oil
    if topic == 'Green Energy':
        sentiment_dict['positive(sentiment_score_tweet)'], sentiment_dict['negative(sentiment_score_tweet)'] = \
            sentiment_dict['negative(sentiment_score_tweet)'], sentiment_dict['positive(sentiment_score_tweet)']
        sentiment_dict['compound(sentiment_score_tweet)'] = -sentiment_dict['compound(sentiment_score_tweet)']
    return sentiment_dict

# DataFrame to store results
res = {}

# Iterate over tweets and calculate sentiment scores
for i, row in tqdm(X_articles.iterrows(), total=len(X_articles)):
    try:
        text = row['tweet.text']
        myid = row['tweet_count']
        
        # Step 1: Get sentiment scores for the tweet
        sentiment_scores = polarity_scores_roberta(text)
        
        # Step 2: Classify the tweet topic
        topic = classify_tweet(row)
        
        # Step 3: Adjust sentiment based on the topic
        sentiment_scores = adjust_sentiment_based_on_topic(sentiment_scores, topic)
        
        # Store results
        res[myid] = sentiment_scores
    except RuntimeError:
        print(f'Broke for id {myid}')

# Convert the results dictionary to a DataFrame
x_articles_results = pd.DataFrame(res).T.reset_index().rename(columns={'index': 'tweet_count'})

# Ensure tweet_count is the same type in both DataFrames
X_articles['tweet_count'] = X_articles['tweet_count'].astype(str)
x_articles_results['tweet_count'] = x_articles_results['tweet_count'].astype(str)

# Merge the new DataFrame with the original one
x_articles_results_final = X_articles.merge(x_articles_results, on='tweet_count', how='left')

# Display the head of the final DataFrame
x_articles_results_final.head()


100%|██████████| 1731/1731 [04:26<00:00,  6.49it/s]


,tweet_count,tweet.created_at,tweet.user.name,tweet.user.location,tweet.user.followers_count,tweet.text,tweet.view_count,tweet.favorite_count,tweet.retweet_count,tweet.reply_count,tweet.media,negative(sentiment_score_tweet),neutral(sentiment_score_tweet),positive(sentiment_score_tweet),compound(sentiment_score_tweet)
0,1,2024-01-02 13:10:06+00:00,EIA,"Washington, DC",181422,#TodayInEnergy - #Brent crude #oilprices averaged $19 per barrel less in 2023 than 2022 https://t.co/M9vt0n6fsn https://t.co/Gexe4zLnDm,20679,30,17,0,"[{'display_url': 'pic.x.com/gexe4zlndm', 'expanded_url': 'https://twitter.com/EIAgov/status/1742171408954736679/photo/1', 'id_str': '1742171381167509506', 'indices': [112, 135], 'media_key': '3_1742171381167509506', 'media_url_https': 'https://pbs.twimg.com/media/GC1vpAhXUAIfxgF.jpg', 'type': 'photo', 'url': 'https://t.co/Gexe4zLnDm', 'ext_media_availability': {'status': 'Available'}, 'features': {'large': {'faces': []}, 'medium': {'faces': []}, 'small': {'faces': []}, 'orig': {'faces': []}}, 'sizes': {'large': {'h': 518, 'w': 1007, 'resize': 'fit'}, 'medium': {'h': 518, 'w': 1007, 'resize': 'fit'}, 'small': {'h': 350, 'w': 680, 'resize': 'fit'}, 'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}}, 'original_info': {'height': 518, 'width': 1007, 'focus_rects': [{'x': 82, 'y': 0, 'w': 925, 'h': 518}, {'x': 489, 'y': 0, 'w': 518, 'h': 518}, {'x': 553, 'y': 0, 'w': 454, 'h': 518}, {'x': 748, 'y': 0, 'w': 259, 'h': 518}, {'x': 0, 'y': 0, 'w': 1007, 'h': 518}]}, 'media_results': {'result': {'media_key': '3_1742171381167509506'}}}]",0.224576,0.714257,0.061168,-0.163408
1,2,2024-01-02 18:11:37+00:00,EIA,"Washington, DC",181422,Reminder: #TodayInEnergy will see some big changes next week.\n\nToday In Energy will remain a critical source for energy analysis and will now be the home for several of our other analysis products. \n\nCheck out what's coming on our website 👇\nhttps://t.co/4ONwfWc9ee,6858,15,10,1,NaN,0.004042,0.304035,0.691923,0.687881
2,3,2024-01-02 19:04:55+00:00,EIA,"Washington, DC",181422,Annual average price of Brent crude #oil: \n\n🛢️ 2023: $83 per barrel \n🛢️ 2022: $101 per barrel\n\n➡️ https://t.co/wZ1Pgfn4tr https://t.co/IAqLWlrDsT,11759,27,9,1,"[{'display_url': 'pic.x.com/iaqlwlrdst', 'expanded_url': 'https://twitter.com/EIAgov/status/1742260703187144753/photo/1', 'id_str': '1742260672396660737', 'indices': [123, 146], 'media_key': '3_1742260672396660737', 'media_url_https': 'https://pbs.twimg.com/media/GC3A2cRWEAEoJ5s.png', 'type': 'photo', 'url': 'https://t.co/IAqLWlrDsT', 'ext_media_availability': {'status': 'Available'}, 'features': {'large': {'faces': []}, 'medium': {'faces': []}, 'small': {'faces': []}, 'orig': {'faces': []}}, 'sizes': {'large': {'h': 426, 'w': 805, 'resize': 'fit'}, 'medium': {'h': 426, 'w': 805, 'resize': 'fit'}, 'small': {'h': 360, 'w': 680, 'resize': 'fit'}, 'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}}, 'original_info': {'height': 426, 'width': 805, 'focus_rects': [{'x': 44, 'y': 0, 'w': 761, 'h': 426}, {'x': 379, 'y': 0, 'w': 426, 'h': 426}, {'x': 431, 'y': 0, 'w': 374, 'h': 426}, {'x': 592, 'y': 0, 'w': 213, 'h': 426}, {'x': 0, 'y': 0, 'w': 805, 'h': 426}]}, 'media_results': {'result': {'media_key': '3_1742260672396660737'}}}]",0.177734,0.775157,0.047109,-0.130625
3,4,2024-01-02 21:09:05+00:00,EIA,"Washington, DC",181422,We're adjusting our data release schedule due to the New Year's holiday on Monday. Check out our release schedule to see how things will change. \n\n👉 https://t.co/GUojmvnlgu https://t.co/nnvWtlnraJ,3671,7,1,1,"[{'display_url': 'pic.x.com/nnvwtlnraj', 'expanded_url': 'https://twitter.com/EIAgov/status/1742291950345027709/photo/1', 'id_str': '1742291929369362432', 'indices': [173, 196], 'media_key': '3_1742291929369362432', 'media_url_https': 'https://pbs.twimg.com/media/GC3dR1kXQAAqgqn.jpg', 'type': 'photo', 'url': 'https://t.co/nnvWtlnraJ', 'ext_media_availability': {'status': 'Available'}, 'features': {'large': {'faces': []}, 'medium': {'faces': []},

In [19]:
# res = {}
# for i, row in tqdm(X_articles.iterrows(), total=len(X_articles)):
#     try:
#         text = row['tweet.text']
#         myid = row['tweet_count']
#         res[myid] = polarity_scores_roberta(text)
#     except RuntimeError:
#         print(f'Broke for id {myid}')

100%|██████████| 1731/1731 [04:17<00:00,  6.71it/s]


In [ ]:
# X_articles_results = pd.DataFrame(res).T
# X_articles_results = X_articles_results.reset_index().rename(columns={'index': 'tweet_count'})
# X_articles_results = X_articles_results.merge(X_articles, how='left')
# X_articles_results.head()

,tweet_count,negative(sentiment_score),neutral(sentiment_score),positive(sentiment_score),compound(sentiment_score),tweet.created_at,tweet.user.name,tweet.user.location,tweet.user.followers_count,tweet.text,tweet.view_count,tweet.favorite_count,tweet.retweet_count,tweet.reply_count,tweet.media
0,1,0.224576,0.714257,0.061168,-0.163408,2024-01-02 13:10:06+00:00,EIA,"Washington, DC",181422,#TodayInEnergy - #Brent crude #oilprices avera...,20679,30,17,0,"[{'display_url': 'pic.x.com/gexe4zlndm', 'expa..."
1,2,0.004042,0.304035,0.691923,0.687881,2024-01-02 18:11:37+00:00,EIA,"Washington, DC",181422,Reminder: #TodayInEnergy will see some big cha...,6858,15,10,1,NaN
2,3,0.177734,0.775157,0.047109,-0.130626,2024-01-02 19:04:55+00:00,EIA,"Washington, DC",181422,Annual average price of Brent crude #oil: \n\n...,11759,27,9,1,"[{'display_url': 'pic.x.com/iaqlwlrdst', 'expa..."
3,4,0.036057,0.877669,0.086274,0.050218,2024-01-02 21:09:05+00:00,EIA,"Washington, DC",181422,We're adjusting our data release schedule due ...,3671,7,1,1,"[{'display_url': 'pic.x.com/nnvwtlnraj', 'expa..."
4,5,0.194202,0.722310,0.083488,-0.110714,2024-01-02 21:30:01+00:00,EIA,"Washington, DC",181422,Comparing September 2023 to September 2022:\n\...,3956,15,3,1,NaN


In [57]:
# Save the merged DataFrame to a new CSV file
output_path = 'X_articles_results.csv'
x_articles_results_final.to_csv(output_path, index=False)

print(f"The merged DataFrame has been saved to {output_path}")

The merged DataFrame has been saved to X_articles_results.csv


In [20]:
# import pandas as pd
# import json

# # Load the CSV file into a DataFrame
# file_path = 'C:\\Users\\aerok\\OneDrive\\Рабочий стол\\Socioeconomic Dynamics\\Sentiment Analysis\\X_articles_results.csv'
# df = pd.read_csv(file_path)

# # Display the first few rows of the DataFrame
# print(df.head())

# # Function to extract media_url_https
# def extract_media_url(media_column):
#     if pd.isna(media_column):
#         return ""
#     try:
#         # Ensure the input is a string
#         media_column = str(media_column)
#         # Parse the JSON-like structure
#         media_items = json.loads(media_column.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
#         if isinstance(media_items, list):
#             # Extract media_url_https from each media item
#             media_urls = [item['media_url_https'] for item in media_items if 'media_url_https' in item]
#             return ', '.join(media_urls)  # Join multiple URLs with a comma
#         else:
#             return ""
#     except (json.JSONDecodeError, TypeError):
#         return ""

# # Apply the extraction function to the tweet.media column
# df['media_url_https'] = df['tweet.media'].apply(extract_media_url)

# # Display the first few rows of the updated DataFrame
# df.tail()




   tweet_count  negative(sentiment_score)  neutral(sentiment_score)  \
0            1                   0.224576                  0.714257   
1            2                   0.004042                  0.304035   
2            3                   0.177734                  0.775157   
3            4                   0.036057                  0.877669   
4            5                   0.194202                  0.722310   

   positive(sentiment_score)  compound(sentiment_score)  \
0                   0.061168                  -0.163408   
1                   0.691923                   0.687881   
2                   0.047109                  -0.130626   
3                   0.086274                   0.050218   
4                   0.083488                  -0.110714   

            tweet.created_at tweet.user.name tweet.user.location  \
0  2024-01-02 13:10:06+00:00             EIA      Washington, DC   
1  2024-01-02 18:11:37+00:00             EIA      Washington, DC   
2  2024-01-02 

,tweet_count,negative(sentiment_score),neutral(sentiment_score),positive(sentiment_score),compound(sentiment_score),tweet.created_at,tweet.user.name,tweet.user.location,tweet.user.followers_count,tweet.text,tweet.view_count,tweet.favorite_count,tweet.retweet_count,tweet.reply_count,tweet.media,media_url_https
1726,1727,0.272725,0.700648,0.026627,-0.246098,2024-07-09 18:17:04+00:00,Asia Fact Check Lab,NaN,148,A claim emerged among Chinese netizens in June...,66,3,1,0,"[{'display_url': 'pic.x.com/znday8bmoj', 'expa...",https://pbs.twimg.com/media/GSEKZmcaUAEbSeh.jpg
1727,1728,0.018073,0.553974,0.427953,0.409881,2024-07-09 18:28:45+00:00,House Appropriations,"Washington, DC",22922,The FY25 Interior and Environment Appropriatio...,1787,9,2,4,"[{'display_url': 'pic.x.com/hwidzcew02', 'expa...",
1728,1729,0.462649,0.478608,0.058743,-0.403906,2024-07-09 18:45:17+00:00,EIA,"Washington, DC",181421,☝️Correction: #Cryptocurrency mining. Apologie...,2245,1,0,0,NaN,
1729,1730,0.004067,0.266414,0.729519,0.725453,2024-07-09 19:46:22+00:00,GetWireless,Minnesota,693,💡100% of the top 20 U.S. oil and gas producers...,26,0,0,0,"[{'display_url': 'pic.x.com/ezsk6nc95w', 'expa...",https://pbs.twimg.com/tweet_video_thumb/GSEbrZ...
1730,1731,0.048669,0.794752,0.156578,0.107909,2024-07-09 19:50:02+00:00,EIA,"Washington, DC",181421,As we continue to highlight the #STEO forecast...,9149,8,3,1,NaN,
